# Integrated Credit Analysis Tool (ICAT)

**Version:** 23.5  
**Status:** Production  

This notebook serves as the interactive quantitative engine for the Adam Platform. It allows analysts to:

1.  **Ingest** raw financials (from XBRL or Manual Input).
2.  **Project** 7-year forward financials based on growth drivers.
3.  **Value** the enterprise using DCF (WACC / Terminal Value).
4.  **Rate** the credit using the SNC Regulatory Mapping logic.
5.  **Visualize** key credit metrics.

In [ ]:

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Configuration
sns.set_theme(style="darkgrid")
plt.rcParams['figure.figsize'] = [12, 6]

print("ICAT Engine Initialized.")

## 1. Financial Ingestion & Projection

In [ ]:
# Assumptions
revenue_base = 1000.0
growth_rate = 0.05
ebitda_margin = 0.25
capex_percent = 0.03
tax_rate = 0.21

years = range(2025, 2032)
data = []

for i, year in enumerate(years):
    rev = revenue_base * ((1 + growth_rate) ** i)
    ebitda = rev * ebitda_margin
    depreciation = rev * 0.02
    ebit = ebitda - depreciation
    taxes = ebit * tax_rate
    nopat = ebit - taxes
    capex = rev * capex_percent
    nwc_change = rev * 0.01
    fcf = nopat + depreciation - capex - nwc_change
    
    data.append({
        "Year": year,
        "Revenue": rev,
        "EBITDA": ebitda,
        "FCF": fcf
    })

df = pd.DataFrame(data)
df.head(7)

## 2. DCF Valuation

In [ ]:
wacc = 0.10
terminal_growth = 0.02

# Discount Periods
df['Discount_Factor'] = [1 / ((1 + wacc) ** i) for i in range(len(df))]
df['PV_FCF'] = df['FCF'] * df['Discount_Factor']

# Terminal Value
last_fcf = df['FCF'].iloc[-1]
terminal_value = (last_fcf * (1 + terminal_growth)) / (wacc - terminal_growth)
pv_terminal_value = terminal_value * df['Discount_Factor'].iloc[-1]

enterprise_value = df['PV_FCF'].sum() + pv_terminal_value

print(f"Enterprise Value: ${enterprise_value:,.2f}")

## 3. SNC Regulatory Rating

In [ ]:
total_debt = 1800.0
lfm_ebitda = df['EBITDA'].iloc[0] # Last Twelve Months (proxy)
leverage = total_debt / lfm_ebitda

print(f"Total Debt: ${total_debt}")
print(f"LTM EBITDA: ${lfm_ebitda:.2f}")
print(f"Leverage Ratio: {leverage:.2f}x")

def get_snc_rating(leverage, coverage=None):
    """Deterministic mapping to SNC Regulatory Ratings."""
    if leverage <= 3.0:
        return "Pass 1 (Strong)"
    elif leverage <= 4.5:
        return "Pass 2 (Good)"
    elif leverage <= 6.0:
        return "Special Mention"
    elif leverage <= 8.0:
        return "Substandard"
    else:
        return "Doubtful"

rating = get_snc_rating(leverage)
print(f"SNC Regulatory Rating: {rating}")

## 4. Visualization

In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(data=df, x='Year', y='FCF', palette='viridis')
plt.title('Projected Free Cash Flow (7-Year)')
plt.ylabel('FCF ($M)')
plt.show()